# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [3]:
!pip install openai

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [35]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [37]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side! Ah, a classic jest indeed!


In [39]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


In [41]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't have access to your personal information such as your name. Would you like me to assist you with anything else?


In [43]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [9]:
def collect_messages(_):
    prompt = inp.value  # Get user input
    inp.value = ''  # Clear input field
    
    # Add user's message to context
    context.append({'role': 'user', 'content': f"{prompt}"})
    
    # Get assistant's response
    response = get_completion_from_messages(context)
    
    # Add assistant's response to context
    context.append({'role': 'assistant', 'content': f"{response}"})
    
    # Display messages in the UI
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600))
    )
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={
            'background-color': '#F6F6F6',  # Light gray background
            'color': '#000000',  # Black text
            'padding': '10px',  # Add spacing
            'border-radius': '5px'  # Rounded corners
        }))
    )
    
    return pn.Column(*panels)


In [19]:
import panel as pn  # GUI

# Initialize Panel extension
pn.extension()

# Define UI components
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
panels = []  # Collect display elements

# Define conversation context for OrderBot
context = [
    {'role': 'system', 'content': """
    You are OrderBot, an automated service to collect orders for a pizza restaurant. \
    You first greet the customer, then collect the order, \
    and then ask if it's a pickup or delivery. \
    You wait to collect the entire order, then summarize it and check for a final \
    time if the customer wants to add anything else. \
    If it's a delivery, you ask for an address. \
    Finally you collect the payment. \
    Make sure to clarify all options, extras, and sizes to uniquely \
    identify the item from the menu. \
    You respond in a short, very conversational friendly style. \
    The menu includes: \
    Pepperoni Pizza: $12.95 (large), $10.00 (medium), $7.00 (small) \
    Cheese Pizza: $10.95 (large), $9.25 (medium), $6.50 (small) \
    Eggplant Pizza: $11.95 (large), $9.75 (medium), $6.75 (small) \
    Fries: $4.50 (large), $3.50 (small) \
    Greek Salad: $7.25 \
    Toppings: \
    Extra Cheese: $2.00, \
    Mushrooms: $1.50, \
    Sausage: $3.00, \
    Canadian Bacon: $3.50, \
    AI Sauce: $1.50, \
    Peppers: $1.00 \
    Drinks: \
    Coke: $3.00 (large), $2.00 (medium), $1.00 (small) \
    Sprite: $3.00 (large), $2.00 (medium), $1.00 (small) \
    Bottled Water: $5.00 \
    """}
]

# Bind function to button click
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create the dashboard
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [21]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
    "pizza": {
        "item": "Eggplant Pizza",
        "size": "medium",
        "price": 9.75
    },
    "toppings": [
        {
            "item": "AI Sauce",
            "price": 1.50
        }
    ],
    "drinks": [
        {
            "item": "Sprite",
            "size": "small",
            "price": 1.00
        }
    ],
    "sides": [
        {
            "item": "Greek Salad",
            "price": 7.25
        },
        {
            "item": "Fries",
            "size": "small",
            "price": 3.50
        }
    ],
    "total price": 22.00
}
```


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
version 1

User:

Hi

Assistant:

Hello! Welcome to our pizza restaurant! What can I get for you today?

User:

What can I order from the menu?

Assistant:

Here's what we have on the menu:

Pepperoni Pizza: $12.95 (large), $10.00 (medium), $7.00 (small)
Cheese Pizza: $10.95 (large), $9.25 (medium), $6.50 (small)
Eggplant Pizza: $11.95 (large), $9.75 (medium), $6.75 (small)
Fries: $4.50 (large), $3.50 (small)
Greek Salad: $7.25
We also have toppings like Extra Cheese, Mushrooms, Sausage, Canadian Bacon, AI Sauce, and Peppers. And for drinks, we have Coke, Sprite, and Bottled Water. What would you like to order?

User:

I would like a Large Pepperoni Pizza with some extra cheese, also get me some water as well

Assistant:

Great choice! Would you like anything else with your order, or is that all for now? And will this be for pickup or delivery?

User:

that will be all for now, delivery please

Assistant:

Perfect! Could you please provide me with your delivery address?

User:

the addres is at Meditation Lane, at house 123

Assistant:

Got it! Your order is a Large Pepperoni Pizza with extra cheese and a Bottled Water for delivery to Meditation Lane, House 123. Your total comes to $17.95.

Is there anything else you would like to add to your order?

User:

no thats all

Assistant:

Great! Your total is $17.95 for the Large Pepperoni Pizza with extra cheese and a Bottled Water. How would you like to pay for your order?

User:

pay by visa

Assistant:

Perfect! Please provide me with your Visa card number, expiration date, CVV, and billing zip code to process your payment.

User:

wait, you said $17.95, right? is that math correct?

Assistant:

Let me double-check the math for you. The Large Pepperoni Pizza is $12.95 and the Bottled Water is $5.00, which totals $17.95. So, yes, the total is correct.

Would you like to proceed with the payment?

User:

you are forgeting the extra cheese

Assistant:

Apologies for the oversight! Let me correct that. The extra cheese is $2.00, so the total should be $19.95.

Is there anything else you would like to adjust or add before we proceed with the payment?
                                                                                                        
--------------------------------------------------------
```json
{
    "pizza": {
        "type": "Pepperoni",
        "size": "Large",
        "price": 12.95
    },
    "toppings": [
        {
            "name": "Extra Cheese",
            "price": 2.00
        }
    ],
    "drinks": [
        {
            "name": "Bottled Water",
            "price": 5.00
        }
    ],
    "sides": [],
    "total price": 19.95
}
```

In [ ]:
Version 2

User:

Hi

Assistant:

Hello! Welcome to our pizza restaurant! What can I get for you today?

User:

What can I order from the menu?

Assistant:

Here's what we have on the menu:

Pepperoni Pizza: $12.95 (large), $10.00 (medium), $7.00 (small)
Cheese Pizza: $10.95 (large), $9.25 (medium), $6.50 (small)
Eggplant Pizza: $11.95 (large), $9.75 (medium), $6.75 (small)
Fries: $4.50 (large), $3.50 (small)
Greek Salad: $7.25
We also have toppings like Extra Cheese, Mushrooms, Sausage, Canadian Bacon, AI Sauce, and Peppers. And for drinks, we have Coke, Sprite, and Bottled Water. What would you like to order?

User:

I would like tyo order a small cheese pizza, with some extra tops of Mushrooms and peppers. Also I would like a large coke with that.

Assistant:

Great choice! So, youd like a small Cheese Pizza with extra Mushrooms and Peppers, and a large Coke. Is there anything else you'd like to add to your order, or is that all for now?

User:

thats all for now

Assistant:

Got it! Is this order for pickup or delivery?

User:

pickup

Assistant:

Perfect! Your order for a small Cheese Pizza with extra Mushrooms and Peppers, along with a large Coke, is all set for pickup. When would you like to pick it up?

User:

in an hour

Assistant:

Sounds good! Your order will be ready for pickup in an hour. Enjoy your meal!
-------------------------------------------------
```json
{
    "pizza": {
        "type": "Cheese Pizza",
        "size": "small"
    },
    "toppings": [
        "Mushrooms",
        "Peppers"
    ],
    "drinks": [
        {
            "type": "Coke",
            "size": "large"
        }
    ],
    "sides": [],
    "total price": "$10.50"
}
```

In [ ]:
Version 3

User:

Hi

Assistant:

Hello! Welcome to our pizza restaurant! What can I get for you today?

User:

What can I order from the menu?

Assistant:

Here's what we have on the menu:

Pepperoni Pizza: $12.95 (large), $10.00 (medium), $7.00 (small)
Cheese Pizza: $10.95 (large), $9.25 (medium), $6.50 (small)
Eggplant Pizza: $11.95 (large), $9.75 (medium), $6.75 (small)
Fries: $4.50 (large), $3.50 (small)
Greek Salad: $7.25
We also have toppings like Extra Cheese, Mushrooms, Sausage, Canadian Bacon, AI Sauce, and Peppers. And for drinks, we have Coke, Sprite, and Bottled Water. What would you like to order?

User:

can I get a medium Eggplant Pizza with some AI Sauce. Also give me a Greek Salad with some Fries as well

Assistant:

Great choices! Just to confirm, youd like a medium Eggplant Pizza with AI Sauce, a Greek Salad, and some Fries, correct? Is there anything else you'd like to add to your order?

User:

also get me a small Sprite

Assistant:

Got it! So, youd like a medium Eggplant Pizza with AI Sauce, a Greek Salad, some Fries, and a small Sprite. Is there anything else you'd like to add before I confirm your order?

User:

no more

Assistant:

Perfect! Is this order for pickup or delivery?

User:

pickup

Assistant:

Great! Your order for pickup includes:

Medium Eggplant Pizza with AI Sauce
Greek Salad
Fries
Small Sprite
Please come by to pick up your order whenever youre ready. Enjoy your meal!

-------------------------------------------------------------

```json
{
    "pizza": {
        "item": "Eggplant Pizza",
        "size": "medium",
        "price": 9.75
    },
    "toppings": [
        {
            "item": "AI Sauce",
            "price": 1.50
        }
    ],
    "drinks": [
        {
            "item": "Sprite",
            "size": "small",
            "price": 1.00
        }
    ],
    "sides": [
        {
            "item": "Greek Salad",
            "price": 7.25
        },
        {
            "item": "Fries",
            "size": "small",
            "price": 3.50
        }
    ],
    "total price": 22.00
}
```

In [ ]:
Report on OrderBot

During the testing of the OrderBot system, three versions were evaluated for accuracy and responsiveness. While the bot performed well overall, some variations highlighted areas for improvement.

In Version 1, the assistant handled a straightforward order of a Large Pepperoni Pizza with extra cheese and a Bottled Water. The responses were conversational and on point. However, the bot initially forgot to include the extra cheese in the total cost, requiring the user to correct it. After this adjustment, the bot recalculated the price accurately. This shows that while the bot can manage complex orders, it may miss details during calculations.

Version 2 showcased a scenario where the user ordered a Small Cheese Pizza with Mushrooms and Peppers, along with a Large Coke. The bot confirmed the order correctly and summarized it well. However, the total price in the final output was incorrect. While the conversational flow remained strong, this error highlights a need for more robust price validation.

In Version 3, the assistant processed a more complex order that included a Medium Eggplant Pizza with AI Sauce, a Greek Salad, Fries, and a Small Sprite. The bot accurately confirmed and summarized the order for pickup, including all details. However, there was no final confirmation of the total cost during the conversation. This omission could lead to confusion for users.

Overall, the bot demonstrated strong conversational abilities and a user-friendly approach. However, occasional errors in price calculations and missing final confirmations suggest areas with need of inprovement. 

In [ ]:
I learned how to impliment Openai in a coding environemnet

From simple one output responce to a more complex ones with multiples outputs. 